<a href="https://colab.research.google.com/github/alikaiser12/AI/blob/main/Area_Marking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from glob import glob
import re
import numpy as np
import cv2
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

# Define the folder path in Google Drive (adjust according to your folder structure)
data_path = '/content/drive/MyDrive/Video'

# List .jpg files in the folder
image_paths = [os.path.join(data_path, file) for file in os.listdir(data_path) if file.lower().endswith('.mp4')]

# Print the list of image paths
print(image_paths)

['/content/drive/MyDrive/Video/video-2 clip-1 (30 sec).mp4', '/content/drive/MyDrive/Video/clip 2.mp4']


In [ ]:
import glob
import re
import os

# Define the folder path in Google Drive
data_path = '/content/drive/MyDrive/Video'

# Use glob to get all jpg files in the folder
# If a filename doesn't have digits, it will be treated as 0 for sorting.
image_paths = sorted(glob.glob(f"{data_path}/*.mp4"), key=lambda x: float(re.findall("(\d+)", x)[0]) if re.findall("(\d+)", x) else 0)

# Print the list of image paths
print(image_paths)

['/content/drive/MyDrive/Video/video-2 clip-1 (30 sec).mp4', '/content/drive/MyDrive/Video/clip 2.mp4']


In [ ]:
import cv2

# Load the video
video_path = "/content/drive/MyDrive/Video/clip 2.mp4"  # Make sure this file is uploaded
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output path
output_path = "/content/marked_video.mp4"
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Define central 40% box boundaries
start_x = int(0.3 * frame_width)  # 50% - 20% = 30%
end_x = int(0.7 * frame_width)    # 50% + 20% = 70%

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Draw green box over the central 40% horizontally
    cv2.rectangle(frame, (start_x, 0), (end_x, frame_height), (0, 255, 0), 3)

    out.write(frame)

# Release resources
cap.release()
out.release()


In [ ]:
from google.colab import files
files.download("/content/marked_video.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import cv2
import numpy as np

# Load the video
video_path = "/content/drive/MyDrive/Video/clip 2.mp4"  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output video writer
output_path = "annotated_obstacle_video.mp4"
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Define the central 40% box
start_x = int(0.3 * frame_width)  # 50% - 20%
end_x = int(0.7 * frame_width)    # 50% + 20%

# Background subtractor for motion detection
fgbg = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50)

# Process video frame-by-frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Apply background subtraction
    fgmask = fgbg.apply(frame)
    _, thresh = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))

    # Find contours from motion mask
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw central green area
    cv2.rectangle(frame, (start_x, 0), (end_x, frame_height), (0, 255, 0), 3)

    # Process each contour (potential obstacle)
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)

        # Ignore small areas (noise)
        if w * h < 500:
            continue

        center_x = x + w // 2

        # Check if the center of the object is within the green box
        if start_x <= center_x <= end_x:
            # Draw red rectangle and add text annotation
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(frame, 'Obstacle Detected', (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Write processed frame to output video
    out.write(frame)

# Release resources
cap.release()
out.release()
# cv2.destroyAllWindows() # Removed as it's not supported in this environment

In [ ]:
from google.colab import files
files.download("/content/annotated_obstacle_video.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Run in Colab or terminal
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17488, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 17488 (delta 0), reused 0 (delta 0), pack-reused 17486 (from 2)
Receiving objects: 100% (17488/17488), 16.55 MiB | 14.46 MiB/s, done.
Resolving deltas: 100% (11995/11995), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 870.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00

In [ ]:
import torch

# Load YOLOv5s (small model) pre-trained on COCO dataset
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf = 0.3  # confidence threshold


/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2025-6-11 Python-3.11.13 torch-2.6.0+cu124 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 180MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
import cv2

# Input/output video
video_path = '/content/drive/MyDrive/Video/clip 2.mp4'
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

output_path = '/content/annotated_yolo_video.mp4'
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

# Define central 40% horizontal area
start_x = int(0.3 * frame_width)
end_x = int(0.7 * frame_width)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO expects RGB
    results = model(frame[..., ::-1])
    detections = results.xyxy[0]  # x1, y1, x2, y2, conf, cls

    # Draw central green area
    cv2.rectangle(frame, (start_x, 0), (end_x, frame_height), (0, 255, 0), 3)

    for *box, conf, cls in detections:
        x1, y1, x2, y2 = map(int, box)
        class_name = model.names[int(cls)]

        center_x = (x1 + x2) // 2
        if start_x <= center_x <= end_x:
            # Draw red bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            label = f"{class_name} ({conf:.2f})"
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    out.write(frame)

cap.release()
out.release()
# cv2.destroyAllWindows() # Removed as it's not supported in this environment where there is no GUI

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.am

In [ ]:
from google.colab import files
files.download(output_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import cv2
import torch
import numpy as np
import os  # Import the os module

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf = 0.3  # Set confidence threshold

# Input/output video
# Ensure this path is correct and accessible in your Google Drive
video_path = '/content/drive/MyDrive/Video/clip 2.mp4'
cap = cv2.VideoCapture(video_path)

# Check if video capture was successful
if not cap.isOpened():
    print(f"Error: Could not open video file {video_path}")
    # You might want to add a way to stop execution or handle this error
else:
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Output with extra height for message box
    output_height = frame_height + 60  # 60px extra for message box
    output_filename = 'output_with_message.mp4' # Define output filename as a variable
    out = cv2.VideoWriter(output_filename,
                          cv2.VideoWriter_fourcc(*'mp4v'), fps,
                          (frame_width, output_height))

    # Define Central 40% box
    start_x = int(0.3 * frame_width)
    end_x = int(0.7 * frame_width)

    def describe_object(class_name):
        # Custom logic per class (you can expand this)
        actions = {
            'person': 'possibly walking',
            'car': 'likely moving',
            'bicycle': 'might be riding',
            'dog': 'possibly walking',
            'truck': 'likely driving',
        }
        return f"{class_name} - {actions.get(class_name, 'in motion')}"

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Run YOLOv5
        results = model(frame[..., ::-1])
        detections = results.xyxy[0]

        # New image with message box space
        annotated_frame = np.zeros((output_height, frame_width, 3), dtype=np.uint8)
        annotated_frame[:frame_height] = frame.copy()

        # Draw green central region
        cv2.rectangle(annotated_frame, (start_x, 0), (end_x, frame_height), (0, 255, 0), 3)

        messages = []

        for *box, conf, cls in detections:
            x1, y1, x2, y2 = map(int, box)
            class_name = model.names[int(cls)]
            center_x = (x1 + x2) // 2

            if start_x <= center_x <= end_x:
                label = f"{class_name} ({conf:.2f})"
                description = describe_object(class_name)
                messages.append(f"Detected: {description}")

                # Draw red box + label
                cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(annotated_frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

        # Draw yellow message box at bottom
        cv2.rectangle(annotated_frame, (0, frame_height),
                      (frame_width, output_height), (0, 255, 255), -1)  # Yellow box

        for idx, msg in enumerate(messages[:2]):  # Display up to 2 messages
            cv2.putText(annotated_frame, msg, (10, frame_height + 30 + idx * 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)  # Black text

        out.write(annotated_frame)

    cap.release()
    out.release()
    # cv2.destroyAllWindows() # Removed as it's not supported in this environment

    # Add a check to see if the file was created successfully
    if os.path.exists(output_filename):
        print(f"Video '{output_filename}' created successfully.")
    else:
        print(f"Error: Video '{output_filename}' was not created.")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-6-11 Python-3.11.13 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning:

Video 'output_with_message.mp4' created successfully.


In [ ]:
from google.colab import files
import os # Import os to check for file existence

output_path_to_download = 'output_with_message.mp4' # Use the same filename as in the previous cell

# Check if the file exists before attempting to download
if os.path.exists(output_path_to_download):
  # Download the processed video
  files.download(output_path_to_download)
else:
  print(f"Cannot download file: {output_path_to_download}. It does not exist.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 🚀 Full Code: Object Detection with Named Labels, Distance Estimation, and Behavior Messages
# ✅ Works in Google Colab

import cv2
import torch
import numpy as np
from google.colab import files
from collections import defaultdict

# Load YOLOv5 model (COCO-trained)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf = 0.3  # Confidence threshold

# Load input video
video_path = '/content/drive/MyDrive/Video/clip 2.mp4'  # Replace if needed
cap = cv2.VideoCapture(video_path)

# Check if video capture was successful
if not cap.isOpened():
    print(f"Error: Could not open video file {video_path}")
    # Handle the error or exit if video not opened
    # Consider adding sys.exit() or raising an exception here
    exit() # Exit the cell execution if video is not opened

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_height = frame_height + 60

# Output video setup
out_path = 'final_annotated_output.mp4'
out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, output_height))

# Check if VideoWriter is opened successfully
if not out.isOpened():
    print(f"Error: Could not create video writer for {out_path}")
    cap.release() # Release the video capture object
    exit() # Exit the cell execution if writer is not opened


# Define central 40% green area
start_x = int(0.3 * frame_width)
end_x = int(0.7 * frame_width)

# For direction tracking
prev_centers = defaultdict(lambda: None)

def estimate_distance(box_height):
    # Simplified distance estimation (assuming same object size)
    # Adjust ref_height for better accuracy based on your scene
    ref_height = 150  # pixels for 1 meter (example value)
    # Avoid division by zero or very small numbers
    if box_height < 5: # Consider small boxes as noise or too far
        return float('inf') # Or some indicator for "very far"
    # Using reciprocal of height scaled by reference height
    return round(ref_height / box_height, 2)


def describe_behavior(prev_x, curr_x, class_name):
    if prev_x is None:
        return f"{class_name} entering"
    # Define a threshold for movement (adjust as needed)
    movement_threshold = 5
    if curr_x > prev_x + movement_threshold:
        return f"{class_name} moving right"
    elif curr_x < prev_x - movement_threshold:
        return f"{class_name} moving left"
    else:
        return f"{class_name} is steady"

frame_id = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        # Break the loop if video reading fails (end of video or error)
        break

    frame_id += 1
    # YOLO expects RGB format (OpenCV reads BGR), so convert frame
    results = model(frame[..., ::-1])
    # results.xyxy[0] contains detections as [x1, y1, x2, y2, conf, cls] for the first image
    detections = results.xyxy[0]

    # Create a new frame with space for messages at the bottom
    annotated_frame = np.zeros((output_height, frame_width, 3), dtype=np.uint8)
    # Copy the current frame into the top part of the new frame
    annotated_frame[:frame_height] = frame.copy()

    # Draw green zone rectangle on the annotated frame
    cv2.rectangle(annotated_frame, (start_x, 0), (end_x, frame_height), (0, 255, 0), 3)

    messages = []
    # Iterate through detected objects
    for i, (*box, conf, cls) in enumerate(detections):
        x1, y1, x2, y2 = map(int, box)
        class_name = model.names[int(cls)]
        center_x = (x1 + x2) // 2
        # center_y = (y1 + y2) // 2 # Not used in this iteration
        height = y2 - y1

        # Estimate distance only if the object is reasonably sized
        distance = estimate_distance(height)
        # Format label with distance or just class name if distance is unreliable
        if distance != float('inf'):
             label = f"{class_name} ({distance}m)"
        else:
             label = class_name # No distance if too small/far

        # Check if the object's center is within the green zone
        in_green = start_x <= center_x <= end_x

        # Describe behavior if the object is in the green zone
        if in_green:
            # Use a unique identifier for tracking if available (like a track ID)
            # For simple frame-by-frame, using index `i` might lead to issues if objects
            # enter/leave detection or change order. A proper tracker is needed for robust
            # object tracking across frames. For this example, we use index `i`.
            prev_x = prev_centers.get(i)
            behavior = describe_behavior(prev_x, center_x, class_name)
            messages.append(f"Detected: {behavior}")
            box_color = (0, 0, 255)  # Red box for objects in the green zone
        else:
            box_color = (255, 0, 0)  # Blue box for objects outside the green zone

        # Store current center_x for the next frame's behavior check
        prev_centers[i] = center_x

        # Draw bounding box and label on the annotated frame
        cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), box_color, 2)
        # Position the text slightly above the bounding box
        cv2.putText(annotated_frame, label, (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, box_color, 2)

    # Draw yellow message box at the bottom of the annotated frame
    cv2.rectangle(annotated_frame, (0, frame_height), (frame_width, output_height), (0, 255, 255), -1) # Yellow color (-1 fills the rectangle)

    # Display up to the first 2 messages in the message box
    for idx, msg in enumerate(messages[:2]):
        # Position text within the message box
        cv2.putText(annotated_frame, msg, (10, frame_height + 30 + idx * 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)  # Black text (0,0,0)

    # Write the fully annotated frame to the output video file
    out.write(annotated_frame)

# Release the video capture and writer objects
cap.release()
out.release()
# cv2.destroyAllWindows() # This line is commented out because it's not supported in Google Colab or headless environments.

# 🔽 Download the generated video from Colab
# First, check if the output file was actually created
import os
if os.path.exists(out_path):
    print(f"Video '{out_path}' created successfully. Downloading...")
    files.download(out_path)
else:
    print(f"Error: Video '{out_path}' was not created. Please check previous errors.")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-6-11 Python-3.11.13 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning:

Video 'final_annotated_output.mp4' created successfully. Downloading...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import cv2
import torch
import numpy as np
import pandas as pd
import json
from google.colab import files
from collections import defaultdict
import os # Import os to check for file existence

# Load YOLOv5
# The line below loads the model. It has been kept as is from your original code.
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf = 0.3

# Input video
video_path = '/content/drive/MyDrive/Video/clip 2.mp4'
cap = cv2.VideoCapture(video_path)

# Check if video capture was successful
if not cap.isOpened():
    print(f"Error: Could not open video file {video_path}")
    # You might want to add a way to stop execution or handle this error
    exit() # Exit the cell execution if video is not opened

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_height = frame_height + 60

# Output video writer
out_path = 'annotated_output_full.mp4'
# Check if VideoWriter is opened successfully
out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, output_height))
if not out.isOpened():
    print(f"Error: Could not create video writer for {out_path}")
    cap.release() # Release the video capture object
    exit() # Exit the cell execution if writer is not opened


# Green area (central 40%)
start_x = int(0.3 * frame_width)
end_x = int(0.7 * frame_width)

# Track previous positions and logs
prev_centers = defaultdict(lambda: None)
logs = []

def estimate_distance(h):
    ref_height = 150
    # Add a small epsilon to avoid division by zero
    return round(ref_height / (h + 1e-5), 2)

def describe_behavior(prev_x, curr_x, name):
    if prev_x is None:
        return f"{name} entered the area"
    # Use a threshold to determine movement
    movement_threshold = 5 # Adjusted threshold for smoother detection
    if curr_x > prev_x + movement_threshold:
        return f"{name} moving right"
    elif curr_x < prev_x - movement_threshold:
        return f"{name} moving left"
    else:
        return f"{name} is steady"

frame_id = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    results = model(frame[..., ::-1])
    detections = results.xyxy[0]

    annotated = np.zeros((output_height, frame_width, 3), dtype=np.uint8)
    annotated[:frame_height] = frame.copy()
    cv2.rectangle(annotated, (start_x, 0), (end_x, frame_height), (0, 255, 0), 3)

    messages = []
    for i, (*box, conf, cls) in enumerate(detections):
        x1, y1, x2, y2 = map(int, box)
        name = model.names[int(cls)]
        center_x = (x1 + x2) // 2
        height = y2 - y1
        # Only estimate distance for objects with some height to avoid noise
        distance = estimate_distance(height) if height > 10 else float('inf')
        label = f"{name} ({distance}m)" if distance != float('inf') else name


        in_green = start_x <= center_x <= end_x
        prev_x = prev_centers.get(i)
        behavior = describe_behavior(prev_x, center_x, name) if in_green else "Outside zone"
        color = (0, 0, 255) if in_green else (255, 0, 0)
        if in_green:
            messages.append(f"Detected: {behavior}")

        # Update previous center position for the current object index
        # Note: Using index 'i' for tracking is basic. A proper object tracker
        # is needed for more robust tracking across frames where object indices might change.
        prev_centers[i] = center_x

        # Draw box and label
        cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2)
        cv2.putText(annotated, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Log the detection
        logs.append({
            "frame": frame_id,
            "name": name,
            "distance": distance,
            "in_green": in_green,
            "box": [x1, y1, x2, y2],
            "behavior": behavior,
            "confidence": float(conf) # Log confidence
        })

    # Message Box
    cv2.rectangle(annotated, (0, frame_height), (frame_width, output_height), (0, 255, 255), -1)
    for idx, msg in enumerate(messages[:2]):
        cv2.putText(annotated, msg, (10, frame_height + 30 + idx * 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

    out.write(annotated)

# Release resources outside the loop after processing is complete
cap.release()
out.release()
# Removed the problematic line as it's not needed in Colab/headless environments.
# cv2.destroyAllWindows()

# Save Logs
df = pd.DataFrame(logs)
df.to_csv("detection_log.csv", index=False)

with open("detection_log.json", "w") as f:
    # Use default=str to handle potential non-serializable objects if needed
    json.dump(logs, f, indent=2)

# Download files
# Add checks to ensure files exist before attempting download
if os.path.exists(out_path):
    files.download(out_path)
else:
    print(f"Error: Output video file not found at {out_path}")

if os.path.exists("detection_log.csv"):
    files.download("detection_log.csv")
else:
    print("Error: CSV log file not found.")

if os.path.exists("detection_log.json"):
    files.download("detection_log.json")
else:
    print("Error: JSON log file not found.")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-6-11 Python-3.11.13 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning:

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Install necessary packages
!pip install -q ultralytics
!pip install -q filterpy
!git clone -q https://github.com/abewley/sort.git


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# Install necessary packages
!pip install -q ultralytics
!pip install -q filterpy

import os # Import os here
import sys # Import sys here

# Clone the sort repository if it's not already there
# Using -q for quiet output
if not os.path.exists('/content/sort'):
    !git clone -q https://github.com/abewley/sort.git
    # Modify sort.py to use 'Agg' backend right after cloning
    # Ensure the path to sort.py is correct
    !sed -i "s/matplotlib.use('TkAgg')/matplotlib.use('Agg')/" /content/sort/sort.py
else:
    print("SORT repository already exists.")
    # Ensure the sed change is applied even if the directory exists
    # This handles cases where a previous run failed before sed
    !sed -i "s/matplotlib.use('TkAgg')/matplotlib.use('Agg')/" /content/sort/sort.py

# Add the sort directory to sys.path BEFORE importing matplotlib or Sort
# This is critical so Python can find the 'sort' module
sys.path.append('/content/sort')

# Import matplotlib and set a non-interactive backend *before* importing Sort
import matplotlib
# Explicitly set the backend, although the sed command above should fix it in sort.py
# Keeping this line as good practice, but the fix needs to be in the imported file.
# This line prevents matplotlib from trying to open a GUI window, which is not supported in Colab.
matplotlib.use('Agg')

# Correct the import statement to directly import Sort from the sort module
# This import should now work because the backend is set to 'Agg' and the path is added.
try:
    # Corrected import statement
    from sort import Sort
except ImportError as e:
    print(f"Failed to import Sort: {e}")
    print("Please ensure the sort directory exists at /content/sort and sort.py was modified correctly.")
    # Exit execution if the critical import fails
    sys.exit("Exiting due to crucial import failure.")


# Load YOLOv5 model
# The line below loads the model. It has been kept as is from your original code.
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.conf = 0.3

# Initialize SORT tracker
# Now that Sort is correctly imported, this line should work.
tracker = Sort()

# %%
# Rest of your code from ipython-input-32-2568382138 onwards
video_path = '/content/drive/MyDrive/Video/clip 2.mp4'  # or upload again if needed
cap = cv2.VideoCapture(video_path)

# Check if video capture was successful
if not cap.isOpened():
    print(f"Error: Could not open video file {video_path}")
    # Handle the error or exit if video not opened
    # Consider adding sys.exit() or raising an exception here
    sys.exit("Exiting because video could not be opened.") # Use sys.exit for clarity


frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_height = frame_height + 60  # for bottom message bar

output_path = 'final_annotated_video.mp4'
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, output_height))

# Check if VideoWriter is opened successfully
if not out.isOpened():
    print(f"Error: Could not create video writer for {output_path}")
    cap.release() # Release the video capture object
    sys.exit("Exiting because video writer could not be created.") # Use sys.exit for clarity

# %%
# Code from ipython-input-34-2568382138 (function definitions and data structures)
from collections import defaultdict
prev_centers = defaultdict(lambda: None)
logs = []

def estimate_distance(h):
    ref_height = 150  # tune for better accuracy
    # Add a small epsilon to avoid division by zero or very small numbers
    # Also, if height is very small, distance is effectively infinite.
    if h < 5: # Consider small boxes as noise or too far
        return float('inf') # Or some indicator for "very far"
    return round(1.0 / (h / ref_height), 2) # Removed the + 1e-5 from denominator as height should be positive


def describe_behavior(prev_x, curr_x, name):
    if prev_x is None:
        return f"{name} entered the area"
    # Use a threshold to determine movement
    movement_threshold = 10 # This threshold seems reasonable based on your code
    if curr_x > prev_x + movement_threshold:
        return f"{name} moving right"
    elif curr_x < prev_x - movement_threshold:
        return f"{name} moving left"
    else:
        return f"{name} is steady"

# %%
# Code from ipython-input-35-2568382138 (main processing loop)
start_x = int(0.3 * frame_width)
end_x = int(0.7 * frame_width)

frame_id = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        # Break the loop if video reading fails (end of video or error)
        break

    frame_id += 1
    # YOLO expects RGB format (OpenCV reads BGR), so convert frame
    results = model(frame[..., ::-1])
    # results.xyxy[0] contains detections as [x1, y1, x2, y2, conf, cls] for the first image
    detections = results.xyxy[0]

    # Prepare detections for SORT tracker: [x1, y1, x2, y2, confidence]
    dets = []
    det_info = {} # Use a dictionary to map detection bounding box to class index
    for i, (*box, conf, cls) in enumerate(detections):
        x1, y1, x2, y2 = map(int, box)
        dets.append([x1, y1, x2, y2, float(conf)])
        # Create a key based on the bounding box for matching later
        # Using rounded values for robustness
        det_key = tuple(map(int, box)) # Keep as int for exact match if needed
        det_info[det_key] = int(cls) # Store class index


    # Convert dets list to numpy array
    # Handle the case where dets is empty (no detections)
    if len(dets) > 0:
        dets = np.array(dets)
        # Update SORT tracker with current detections
        tracked_objects = tracker.update(dets)
    else:
        # If no detections, update tracker with empty array and no new detections
        tracked_objects = tracker.update(np.empty((0,5))) # Pass empty array with 5 columns (x1, y1, x2, y2, conf)


    # Create a blank frame for annotation
    annotated = np.zeros((output_height, frame_width, 3), dtype=np.uint8)
    # Copy the original frame into the top part
    annotated[:frame_height] = frame.copy()

    # Draw the green central zone
    cv2.rectangle(annotated, (start_x, 0), (end_x, frame_height), (0, 255, 0), 3)

    messages = [] # List to store messages for the bottom bar

    # Iterate through the objects returned by the tracker
    for obj in tracked_objects:
        # Tracker returns [x1, y1, x2, y2, track_id]
        # Ensure track_id is an integer
        x1, y1, x2, y2, track_id = map(int, obj)

        # Calculate properties from tracked box
        center_x = (x1 + x2) // 2
        # center_y = (y1 + y2) // 2 # Not used in this iteration
        height = y2 - y1

        # Estimate distance
        distance = estimate_distance(height)

        # Try to find the class name using the original detection info
        # Find the closest matching detection box for the tracked object's box
        matched_class_name = "Unknown"
        closest_match_dist = float('inf')
        # Initialize confidence for logging
        matched_conf = None

        # Iterate through original detections to find a match based on proximity
        # We iterate through the original `detections` list which has confidence and class
        # rather than the `det_info` dictionary which only has class.
        for *orig_box, conf, cls in detections:
            orig_x1, orig_y1, orig_x2, orig_y2 = map(int, orig_box)
            # Simple Euclidean distance between top-left corners as a proxy for matching
            dist = np.sqrt((orig_x1 - x1)**2 + (orig_y1 - y1)**2)
            # Use a slightly larger threshold or IOU for better matching
            # For simplicity, keeping the distance check, but increase tolerance
            if dist < closest_match_dist and dist < 30: # Reduced threshold for matching distance
                 closest_match_dist = dist
                 matched_class_name = model.names[int(cls)]
                 matched_conf = float(conf)


        name = matched_class_name # Use the matched class name


        # Format the label with ID and distance (if available)
        label = f"{name} ID:{track_id}"
        if distance != float('inf'):
             label += f" ({distance}m)"


        # Check if the object's center is within the green zone
        in_green = start_x <= center_x <= end_x

        # Describe behavior based on movement using the track ID
        prev_x = prev_centers.get(track_id) # Get previous center using the track ID
        behavior = describe_behavior(prev_x, center_x, name) if in_green else "Outside zone"

        # Store current center_x using the track ID for the next frame's behavior check
        prev_centers[track_id] = center_x

        # Determine bounding box color based on location
        color = (0, 0, 255) if in_green else (255, 0, 0) # Red if in green zone, Blue otherwise


        # If the object is in the green zone, add its behavior to the messages
        if in_green:
             # Add a more descriptive message including object ID and name
            messages.append(f"ID:{track_id} ({name}) {behavior}")


        # Draw the bounding box and label on the annotated frame
        cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2)
        # Position the text slightly above the bounding box
        cv2.putText(annotated, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Log the tracking and detection information
        logs.append({
            "frame": frame_id,
            "object_id": int(track_id),
            "name": name,
            "distance": distance if distance != float('inf') else None, # Log None if distance is inf
            "in_green": in_green,
            "box": [x1, y1, x2, y2],
            "behavior": behavior,
            "confidence": matched_conf # Log the confidence of the matched detection
        })


    # Draw the yellow message box at the bottom
    # Check if there are any messages to display before drawing the box
    if messages:
        cv2.rectangle(annotated, (0, frame_height), (frame_width, output_height), (0, 255, 255), -1) # Yellow color (-1 fills the rectangle)

        # Display up to the first 2 messages in the message box
        for idx, msg in enumerate(messages[:2]):
            # Position text within the message box
            cv2.putText(annotated, msg, (10, frame_height + 30 + idx * 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)  # Black text (0,0,0)

    # Write the fully annotated frame to the output video file
    out.write(annotated)

# Release the video capture and writer objects outside the loop
cap.release()
out.release()
# cv2.destroyAllWindows() # This line is commented out because it's not supported in Google Colab or headless environments.

SORT repository already exists.


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-6-11 Python-3.11.13 torch-2.6.0+cu124 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 111MB/s] 

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralyt

In [ ]:
video_path = '/content/drive/MyDrive/Video/clip 2.mp4'  # or upload again if needed
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_height = frame_height + 60  # for bottom message bar

output_path = 'final_annotated_video.mp4'
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, output_height))


In [ ]:
from collections import defaultdict
prev_centers = defaultdict(lambda: None)
logs = []

def estimate_distance(h):
    ref_height = 150  # tune for better accuracy
    return round(1.0 / (h / ref_height + 1e-5), 2)

def describe_behavior(prev_x, curr_x, name):
    if prev_x is None:
        return f"{name} entered the area"
    if curr_x > prev_x + 10:
        return f"{name} moving right"
    elif curr_x < prev_x - 10:
        return f"{name} moving left"
    else:
        return f"{name} is steady"


In [ ]:
start_x = int(0.3 * frame_width)
end_x = int(0.7 * frame_width)

frame_id = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    results = model(frame[..., ::-1])
    detections = results.xyxy[0]

    dets = []
    det_info = []
    for i, (*box, conf, cls) in enumerate(detections):
        x1, y1, x2, y2 = map(int, box)
        dets.append([x1, y1, x2, y2, float(conf)])
        det_info.append((x1, y1, x2, y2, int(cls)))

    dets = np.array(dets)
    tracked_objects = tracker.update(dets)

    annotated = np.zeros((output_height, frame_width, 3), dtype=np.uint8)
    annotated[:frame_height] = frame.copy()
    cv2.rectangle(annotated, (start_x, 0), (end_x, frame_height), (0, 255, 0), 3)

    messages = []

    for obj in tracked_objects:
        x1, y1, x2, y2, track_id = map(int, obj)
        center_x = (x1 + x2) // 2
        height = y2 - y1
        distance = estimate_distance(height)

        # Match detection to get class name
        match = next((box for box in det_info if abs(box[0] - x1) < 10 and abs(box[1] - y1) < 10), None)
        if not match:
            continue

        name = model.names[match[4]]
        label = f"{name} ID:{track_id} ({distance}m)"
        in_green = start_x <= center_x <= end_x
        prev_x = prev_centers[track_id]
        behavior = describe_behavior(prev_x, center_x, name) if in_green else "Outside zone"
        prev_centers[track_id] = center_x
        color = (0, 0, 255) if in_green else (255, 0, 0)

        cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2)
        cv2.putText(annotated, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        if in_green:
            messages.append(behavior)

        logs.append({
            "frame": frame_id,
            "object_id": int(track_id),
            "name": name,
            "distance": distance,
            "in_green": in_green,
            "box": [x1, y1, x2, y2],
            "behavior": behavior
        })

    # Yellow message bar
    cv2.rectangle(annotated, (0, frame_height), (frame_width, output_height), (0, 255, 255), -1)
    for idx, msg in enumerate(messages[:2]):
        cv2.putText(annotated, msg, (10, frame_height + 30 + idx * 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

    out.write(annotated)

cap.release()
out.release()
cv2.destroyAllWindows()


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.am

error: OpenCV(4.11.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'


In [ ]:
# Code from ipython-input-6-3220531614 (main processing loop)
start_x = int(0.3 * frame_width)
end_x = int(0.7 * frame_width)

frame_id = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    results = model(frame[..., ::-1])
    detections = results.xyxy[0]

    dets = []
    det_info = []
    for i, (*box, conf, cls) in enumerate(detections):
        x1, y1, x2, y2 = map(int, box)
        dets.append([x1, y1, x2, y2, float(conf)])
        det_info.append((x1, y1, x2, y2, int(cls)))

    dets = np.array(dets)
    tracked_objects = tracker.update(dets)

    annotated = np.zeros((output_height, frame_width, 3), dtype=np.uint8)
    annotated[:frame_height] = frame.copy()
    cv2.rectangle(annotated, (start_x, 0), (end_x, frame_height), (0, 255, 0), 3)

    messages = []

    for obj in tracked_objects:
        x1, y1, x2, y2, track_id = map(int, obj)
        center_x = (x1 + x2) // 2
        height = y2 - y1
        distance = estimate_distance(height)

        # Match detection to get class name
        # A more robust matching could use Intersection over Union (IOU)
        # For simplicity, keeping the proximity check as in the original code
        match = next((box for box in det_info if abs(box[0] - x1) < 10 and abs(box[1] - y1) < 10), None)
        if not match:
            # If no close detection match is found for a tracked object,
            # perhaps skip annotation or use a default name/behavior.
            # For now, continue to the next tracked object.
            continue

        name = model.names[match[4]]
        # Include distance in the label only if it's a valid number
        label = f"{name} ID:{track_id}"
        if distance != float('inf'):
             label += f" ({distance}m)"

        in_green = start_x <= center_x <= end_x

        # Ensure track_id exists in prev_centers before accessing it
        prev_x = prev_centers.get(track_id)
        # Only describe behavior if the object is currently in the green zone
        behavior = describe_behavior(prev_x, center_x, name) if in_green else "Outside zone"

        # Update the previous center position using the unique track ID
        prev_centers[track_id] = center_x

        # Determine box color based on whether the object is in the green zone
        color = (0, 0, 255) if in_green else (255, 0, 0)

        # Draw bounding box and label
        cv2.rectangle(annotated, (x1, y1), (x2, y2), color, 2)
        # Position label slightly above the box
        cv2.putText(annotated, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Add message to the message list only if the object is in the green zone
        if in_green:
            # Add object ID and name to the message for clarity
            messages.append(f"ID:{track_id} ({name}) {behavior}")


        # Log the information for this tracked object in this frame
        logs.append({
            "frame": frame_id,
            "object_id": int(track_id),
            "name": name,
            # Log distance as None if it's infinite
            "distance": distance if distance != float('inf') else None,
            "in_green": in_green,
            "box": [x1, y1, x2, y2],
            "behavior": behavior,
            # Note: Confidence from original detection is not readily available here
            # after tracking. If needed, you'd need to store it during the matching step.
            # For now, it's omitted or can be added if you refine the matching.
            # "confidence": matched_conf # Add this if you can reliably get confidence
        })


    # Draw the yellow message bar at the bottom ONLY if there are messages to display
    if messages:
        cv2.rectangle(annotated, (0, frame_height), (frame_width, output_height), (0, 255, 255), -1) # Yellow color (-1 fills the rectangle)
        # Display up to the first 2 messages
        for idx, msg in enumerate(messages[:2]):
            cv2.putText(annotated, msg, (10, frame_height + 30 + idx * 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2) # Black text

    # Write the frame to the output video
    out.write(annotated)

# Release the video capture and writer objects AFTER the loop finishes
cap.release()
out.release()
# Remove the line that caused the error
# cv2.destroyAllWindows()

In [ ]:
# Save logs
df = pd.DataFrame(logs)
df.to_csv("final_detection_log.csv", index=False)
with open("final_detection_log.json", "w") as f:
    json.dump(logs, f, indent=2)

# Download results
files.download("final_annotated_video.mp4")
files.download("final_detection_log.csv")
files.download("final_detection_log.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>